### Install wikipedia and lxml

In [1]:
!conda install -c conda-forge wikipedia --yes
!conda install -c conda-forge lxml --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |   py36h9f0ad1d_0         160 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.9.2         |   py36h45558ae_0         613 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-forge
    soupsieve-1.9.4            |   py36h9f0ad1d_1          58 KB  conda-forge
    urllib3-1.25.9             |             py_0          92 KB  conda-forge
    wikipedia-1.4.0            |             py_2          13 KB  conda-forge
    

### Get the html source just using pandas read_html

In [2]:
import pandas as pd
import wikipedia as wp
import numpy as np

html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]

### STEP 1 - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [3]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### STEP 2 - Ignore cells with a borough that is Not assigned

In [4]:
print("Before: ", df.shape)
df = df[df.Borough != 'Not assigned']
# after drop reset your index
df.reset_index(drop=True,inplace=True)
print("After: ", df.shape)

Before:  (180, 3)
After:  (103, 3)


### STEP 3 - Rows will be combined into one row with the neighborhoods separated with a comma
### There is no need to combine rows as the last version of wikipedia has unique values

In [5]:
df['Neighborhood']=df['Neighborhood'].apply(lambda x: x.replace(' / ', ', '))
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### STEP 4 - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
### There is no such cell in the last version of wikipedia

In [6]:
df.shape

(103, 3)

In [7]:
### STEP 5 - Read csv file

In [16]:
path = 'https://cocl.us/Geospatial_data'
gd = pd.DataFrame(pd.read_csv(path))
gd.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
t=df.merge(gd,left_on='Postal code',right_on='Postal Code')

In [18]:
t.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


In [19]:
t = t.drop(columns=['Postal Code'])
t = t.rename(columns={"Postal code": "PostalCode"})
t.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [21]:
t.shape

(103, 5)